In [2]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib as plt
import seaborn as sns

# 1) Paths and existence check
DATA_PATH = r"C:\Users\habib\OneDrive\المستندات\Graduation Project\GRAD-proj-DEPI\DS1\Data\Raw\heart_disease_uci.csv"  
assert os.path.exists(DATA_PATH), f"File not found: {DATA_PATH}"

# 2) Load dataset
df = pd.read_csv(DATA_PATH)

# 3) Inspect shape, columns, and dtypes (before rename)
print("Shape:", df.shape)
print("\nOriginal columns:\n", list(df.columns))
print("\nData types:\n", df.dtypes)

Shape: (920, 16)

Original columns:
 ['id', 'age', 'sex', 'dataset', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalch', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num']

Data types:
 id            int64
age           int64
sex          object
dataset      object
cp           object
trestbps    float64
chol        float64
fbs          object
restecg      object
thalch      float64
exang        object
oldpeak     float64
slope        object
ca          float64
thal         object
num           int64
dtype: object


In [3]:
# 4) Define friendly column names mapping
friendly_map = {
    "id": "Record ID",
    "age": "Age (years)",
    "sex": "Sex",
    "dataset": "Data Source",
    "cp": "Chest Pain Type",
    "trestbps": "Resting BP (mm Hg)",
    "chol": "Cholesterol (mg/dl)",
    "fbs": "Fasting Blood Sugar",
    "restecg": "Resting ECG",
    "thalch": "Max Heart Rate (bpm)",
    "exang": "Exercise Angina",
    "oldpeak": "ST Depression (oldpeak)",
    "slope": "ST Slope",
    "ca": "Major Vessels (0–3)",
    "thal": "Thalassemia",
    "num": "Heart Disease Class (0–4)"
}

# Only rename keys that exist to avoid errors
friendly_map_existing = {k: v for k, v in friendly_map.items() if k in df.columns}

# 5) Apply rename
df = df.rename(columns=friendly_map_existing)

# 6) Confirm new header and dtypes
print("\nRenamed columns:\n", list(df.columns))
print("\nData types after rename (types don’t change by rename):\n", df.dtypes)


Renamed columns:
 ['Record ID', 'Age (years)', 'Sex', 'Data Source', 'Chest Pain Type', 'Resting BP (mm Hg)', 'Cholesterol (mg/dl)', 'Fasting Blood Sugar', 'Resting ECG', 'Max Heart Rate (bpm)', 'Exercise Angina', 'ST Depression (oldpeak)', 'ST Slope', 'Major Vessels (0–3)', 'Thalassemia', 'Heart Disease Class (0–4)']

Data types after rename (types don’t change by rename):
 Record ID                      int64
Age (years)                    int64
Sex                           object
Data Source                   object
Chest Pain Type               object
Resting BP (mm Hg)           float64
Cholesterol (mg/dl)          float64
Fasting Blood Sugar           object
Resting ECG                   object
Max Heart Rate (bpm)         float64
Exercise Angina               object
ST Depression (oldpeak)      float64
ST Slope                      object
Major Vessels (0–3)          float64
Thalassemia                   object
Heart Disease Class (0–4)      int64
dtype: object


In [4]:
# 7) Quick peek of first rows
print("\nHead preview:")
print(df.head(10).to_string(index=False))

# 8) Save an explicit Phase 1 snapshot with friendly headers
out_path = "phase1_friendly_headers.csv"
df.to_csv(out_path, index=False)
print(f"\nSaved Phase 1 snapshot to: {out_path}")


Head preview:
 Record ID  Age (years)    Sex Data Source Chest Pain Type  Resting BP (mm Hg)  Cholesterol (mg/dl) Fasting Blood Sugar    Resting ECG  Max Heart Rate (bpm) Exercise Angina  ST Depression (oldpeak)    ST Slope  Major Vessels (0–3)       Thalassemia  Heart Disease Class (0–4)
         1           63   Male   Cleveland  typical angina               145.0                233.0                True lv hypertrophy                 150.0           False                      2.3 downsloping                  0.0      fixed defect                          0
         2           67   Male   Cleveland    asymptomatic               160.0                286.0               False lv hypertrophy                 108.0            True                      1.5        flat                  3.0            normal                          2
         3           67   Male   Cleveland    asymptomatic               120.0                229.0               False lv hypertrophy                 129.0 

In [5]:
# Define essential, useful columns
ESSENTIAL_NUM = [
    "Age (years)", "Resting BP (mm Hg)", "Cholesterol (mg/dl)",
    "Max Heart Rate (bpm)", "ST Depression (oldpeak)", "Major Vessels (0–3)"
]
ESSENTIAL_CAT = [
    "Chest Pain Type", "Resting ECG", "ST Slope", "Thalassemia", "Sex"
]
ESSENTIAL_BIN = [
    "Fasting Blood Sugar", "Exercise Angina"
]
TARGET = "Heart Disease Class (0–4)"


In [6]:
# Columns allowed to remain only for auditing (not for modeling)
AUDIT_ONLY = ["Data Source"]  # keep for site QA; exclude from features to avoid leakage


In [7]:
#Identify unneeded columns to drop (e.g., Record ID, any extras not in the above sets)
friendly_sets = set(ESSENTIAL_NUM + ESSENTIAL_CAT + ESSENTIAL_BIN + [TARGET] + AUDIT_ONLY)
existing_cols = set(df.columns)
drop_candidates = sorted([c for c in existing_cols if c not in friendly_sets])

# Drop truly irrelevant columns now
df_clean = df.drop(columns=drop_candidates, errors="ignore").copy()
print("Dropped irrelevant columns:", drop_candidates)


Dropped irrelevant columns: ['Record ID']


In [8]:
# Remove duplicates
before = len(df_clean)
df_clean = df_clean.drop_duplicates()
after = len(df_clean)
print(f"Duplicates removed: {before - after} (from {before} to {after})")

Duplicates removed: 2 (from 920 to 918)


In [9]:
#Standardize binary columns to 0/1 (TRUE/FALSE -> 1/0; keep numeric if already numeric)
def to_01(s):
    su = s.astype(str).str.upper()
    mapped = su.map({"TRUE":1, "FALSE":0})
    return pd.to_numeric(mapped.where(su.isin(["TRUE","FALSE"]), s), errors="coerce")

for b in ESSENTIAL_BIN:
    if b in df_clean.columns:
        df_clean[b] = to_01(df_clean[b])


In [10]:
#Ensure numeric dtypes where appropriate
for c in ESSENTIAL_NUM + [TARGET]:
    if c in df_clean.columns:
        df_clean[c] = pd.to_numeric(df_clean[c], errors="coerce")


In [11]:
#Visualize missing values BEFORE imputation
miss_before = df_clean.isna().sum().reset_index()
miss_before.columns = ["Column","Missing"]
fig_before = px.bar(miss_before, x="Column", y="Missing", text="Missing",
                    title="Missing Values (Before Imputation)")
fig_before.update_traces(textposition="outside",
                         hovertemplate="%{x}: %{y} missing<extra></extra>")
fig_before.update_layout(xaxis_tickangle=45, plot_bgcolor="white", paper_bgcolor="white")
fig_before.show()


In [12]:
# Apply safe missing handling

#This keeps your original clean data untouched while experimenting.
df_work = df_clean.copy()

# 1) Categorical high-missing -> add 'Unknown'
hi_miss_cat = ["ST Slope", "Thalassemia", "Resting ECG", "Chest Pain Type"]
for c in hi_miss_cat:
    if c in df_work.columns:
        df_work[c] = df_work[c].astype("category")
        if df_work[c].isna().any():
            df_work[c] = df_work[c].cat.add_categories(["Unknown"]).fillna("Unknown")

# 2) Major Vessels: prefer categorical buckets with 'Unknown'
if "Major Vessels (0–3)" in df_work.columns:
    ca_str = df_work["Major Vessels (0–3)"].astype("Int64").astype(str)
    df_work["Major Vessels (0–3)"] = ca_str.replace({"<NA>":"Unknown","nan":"Unknown","NaN":"Unknown"})
    df_work["Major Vessels (0–3)"] = df_work["Major Vessels (0–3)"].astype("category")

# 3) Numeric median + missing flags
num_cols = ["Age (years)", "Resting BP (mm Hg)", "Cholesterol (mg/dl)",
            "Max Heart Rate (bpm)", "ST Depression (oldpeak)"]
for c in num_cols:
    if c in df_work.columns:
        df_work[f"{c} Missing"] = df_work[c].isna().astype(int)
        med = df_work[c].median(skipna=True)
        df_work[c] = df_work[c].fillna(med)

# 4) Binary leave NaN + flag (already 0/1 where known)
bin_cols = ["Fasting Blood Sugar", "Exercise Angina"]
for c in bin_cols:
    if c in df_work.columns:
        df_work[f"{c} Missing"] = df_work[c].isna().astype(int)
        # do not fill NaN with 0 here

# 5) Optionally drop rows that miss > half of core fields
core = num_cols + ["Chest Pain Type","Resting ECG","ST Slope","Thalassemia","Major Vessels (0–3)","Fasting Blood Sugar","Exercise Angina"]
mask_too_many_missing = df_work[core].isna().sum(axis=1) > (len(core) // 2)
dropped = int(mask_too_many_missing.sum())
df_work = df_work.loc[~mask_too_many_missing].copy()
print(f"Dropped rows with too many missing core fields: {dropped}")

# Save version after safe handling
df_work.to_csv("phase2_missing_handled_safe.csv", index=False)
print("Saved: phase2_missing_handled_safe.csv")


Dropped rows with too many missing core fields: 0
Saved: phase2_missing_handled_safe.csv


In [13]:
# Ensure FBS and EA are 0/1 where known, keep NaN, add missing flags

def to_01_soft(s):
    su = s.astype(str).str.upper()
    mapped = su.map({"TRUE":1, "FALSE":0, "1":1, "0":0})
    # Use mapped where it matches, else try numeric; leave others NaN
    out = pd.to_numeric(mapped.where(su.isin(["TRUE","FALSE","1","0"]), s), errors="coerce")
    return out

for col in ["Fasting Blood Sugar", "Exercise Angina"]:
    if col in df_work.columns:
        df_work[col] = to_01_soft(df_work[col])
        df_work[f"{col} Missing"] = df_work[col].isna().astype(int)

print("Remaining NaNs:")
print(df_work[["Fasting Blood Sugar","Exercise Angina",
               "Fasting Blood Sugar Missing","Exercise Angina Missing"]].isna().sum())

df_work.to_csv("phase2_model_ready_with_binary_missing_flags.csv", index=False)
print("Saved: phase2_model_ready_with_binary_missing_flags.csv")


Remaining NaNs:
Fasting Blood Sugar            90
Exercise Angina                55
Fasting Blood Sugar Missing     0
Exercise Angina Missing         0
dtype: int64
Saved: phase2_model_ready_with_binary_missing_flags.csv


In [14]:

miss_after = df_work.isna().sum().reset_index()
miss_after.columns = ["Column","Missing"]

fig_miss_after = px.bar(
    miss_after, x="Column", y="Missing", text="Missing",
    title="Missing Values After Safe Handling"
)
fig_miss_after.update_traces(
    textposition="outside",
    hovertemplate="%{x}: %{y} missing<extra></extra>"
)
fig_miss_after.update_layout(
    xaxis_tickangle=45, plot_bgcolor="white", paper_bgcolor="white"
)
fig_miss_after.show()

In [15]:
check_cols = [
    "ST Slope", "Thalassemia", "Resting ECG", "Chest Pain Type", "Major Vessels (0–3)",
    "Fasting Blood Sugar", "Exercise Angina",
    "Fasting Blood Sugar Missing", "Exercise Angina Missing"
]
for c in check_cols:
    if c in df_work.columns:
        na = int(df_work[c].isna().sum())
        uniq = int(df_work[c].nunique(dropna=True))
        print(f"{c:28s} | NaN: {na:4d} | Unique (non-NaN): {uniq}")


ST Slope                     | NaN:    0 | Unique (non-NaN): 4
Thalassemia                  | NaN:    0 | Unique (non-NaN): 4
Resting ECG                  | NaN:    0 | Unique (non-NaN): 4
Chest Pain Type              | NaN:    0 | Unique (non-NaN): 4
Major Vessels (0–3)          | NaN:    0 | Unique (non-NaN): 5
Fasting Blood Sugar          | NaN:   90 | Unique (non-NaN): 2
Exercise Angina              | NaN:   55 | Unique (non-NaN): 2
Fasting Blood Sugar Missing  | NaN:    0 | Unique (non-NaN): 2
Exercise Angina Missing      | NaN:    0 | Unique (non-NaN): 2


In [16]:
import json, time

# 1) Full snapshot
df_work.to_csv("phase2_final_full.csv", index=False)
print("Saved: phase2_final_full.csv")

# 2) Modeling-ready (exclude Data Source, keep flags)
exclude = {"Data Source"}
model_cols = [c for c in df_work.columns if c not in exclude]
df_model_ready = df_work[model_cols].copy()
df_model_ready.to_csv("phase2_final_model_ready.csv", index=False)
print("Saved: phase2_final_model_ready.csv")

# 3) Short log/summary
summary = {
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "rows": int(df_work.shape[0]),
    "cols": int(df_work.shape[1]),
    "kept_features_note": "Clinical signals and demographics retained; Data Source excluded from modeling to avoid leakage.",
    "missing_strategy": {
        "categorical_high_missing": "Added 'Unknown' level",
        "major_vessels": "Categorical buckets 0/1/2/3 + 'Unknown'",
        "numeric": "Median fill + Missing flag",
        "binary_flags": "Left NaN + Missing flag; no forced zeros"
    }
}
with open("phase2_summary.json", "w") as f:
    json.dump(summary, f, indent=2)
print("Saved: phase2_summary.json")


Saved: phase2_final_full.csv
Saved: phase2_final_model_ready.csv
Saved: phase2_summary.json


In [17]:
# Final verification for modeling readiness (no forced fills for FBS/EA)
must_have = [
    "Age (years)", "Resting BP (mm Hg)", "Cholesterol (mg/dl)",
    "Max Heart Rate (bpm)", "ST Depression (oldpeak)",
    "Chest Pain Type", "Resting ECG", "ST Slope", "Thalassemia",
    "Major Vessels (0–3)", "Fasting Blood Sugar", "Exercise Angina",
    "Fasting Blood Sugar Missing", "Exercise Angina Missing",
    "Heart Disease Class (0–4)"
]
missing_cols = [c for c in must_have if c not in df_model_ready.columns]
print("Missing essential columns:", missing_cols)

# Sanity: confirm Unknown is present for the categorical set
for c in ["ST Slope","Thalassemia","Resting ECG","Chest Pain Type","Major Vessels (0–3)"]:
    if c in df_model_ready.columns:
        has_unknown = (df_model_ready[c].astype(str) == "Unknown").any()
        print(f"{c}: Unknown present? {has_unknown}")


Missing essential columns: []
ST Slope: Unknown present? True
Thalassemia: Unknown present? True
Resting ECG: Unknown present? True
Chest Pain Type: Unknown present? False
Major Vessels (0–3): Unknown present? True


In [18]:
# Make Chest Pain Type consistent: add 'Unknown' if NaNs exist
col = "Chest Pain Type"
if col in df_model_ready.columns:
    if df_model_ready[col].isna().any():
        df_model_ready[col] = df_model_ready[col].astype("category")
        df_model_ready[col] = df_model_ready[col].cat.add_categories(["Unknown"]).fillna("Unknown")
        print("Added 'Unknown' to Chest Pain Type where NaN existed.")
    else:
        print("Chest Pain Type has no NaNs; no change needed.")


Chest Pain Type has no NaNs; no change needed.
